<p hidden>$
\newcommand{\phm}{\phantom{-}}
\newcommand{\vb}{\underline{\mathbf{b}}}
\newcommand{\vf}{\underline{\mathbf{f}}}
\newcommand{\vk}{\underline{\mathbf{k}}}
\newcommand{\vx}{\underline{\mathbf{x}}}
\newcommand{\vy}{\underline{\mathbf{y}}}
\newcommand{\deriv}[3][]{\frac{\mathrm{d}^{#1}#2}{\mathrm{d}#3^{#1}}}
\newcommand{\partderiv}[3][]{\frac{\partial^{#1}#2}{\partial#3^{#1}}}
\newcommand{\intd}{\,\mathrm{d}}
\newcommand{\rmd}{\mathrm{d}}
\DeclareMathOperator{\Uniform}{Uniform}
\DeclareMathOperator{\Poisson}{Poisson}
\DeclareMathOperator{\Normal}{Normal}
\DeclareMathOperator{\Exponential}{Exponential}
\DeclareMathOperator{\GammaDist}{Gamma}
\DeclareMathOperator{\Prob}{P}
\DeclareMathOperator{\Exp}{E}
\DeclareMathOperator{\Var}{Var}
$</p>

# Lab 3: Linear Systems

### Topics

- **Mathematics:** Gaussian elimination; limitations of this basic method.
- **Python:** Numpy array indexing, writing code for row operations, back substitution and basic partial pivoting.

In [1]:
import numpy as np

## Basic Gaussian elimination

Gaussian elimination is the basic method for solving linear systems by row operations.  For example, consider the linear system $A\vx = \vb$ where

$$
  A = \begin{bmatrix}
        2 & \phm4 & -1 \\
        2 & \phm2 & \phm8 \\
        1 & \phm1 & \phm9 \\
      \end{bmatrix}, \qquad
  \vb = \begin{bmatrix} -2 \\ \phm1 \\ \phm0 \end{bmatrix}.
$$

**Maths usually numbers the rows 1, 2, 3, but Python numbers them 0, 1, 2.  In this lab we will number the rows 0, 1, 2 in the maths as well, to be consistent.**  The first row operation in the Gaussian elimination process for this system eliminates the 2 from the second row (i.e. row 1): $R_1 \leftarrow R_1 - R_0$.

Enter the matrix $A$ and column vector $\vb$ as defined above into Python below.  **Be careful about the data type of the arrays**: `np.array([[1, 2], [3, 4]])` will make an array of `int`s, which will cause problems later.  To get an array of `float`s, you need either to have one or more `float`s in the list you pass in, e.g. `np.array([[1., 2.], [3., 4.]])`, or to specify the array data type: `np.array([[1, 2], [3, 4]], dtype=float)`.

It's a bit simpler to have $\vb$ as a 1D array `np.array([a, b, c])` rather than a 2D single-column matrix `np.array([[a], [b], [c]])`, but 2D single-column matrices print out nicer.  Either will work as long as you are consistent.

Try typing in commands that do the row operation $R_1 \leftarrow R_1 - R_0$.  Remember to do the row operation on the right-hand side vector $\vb$ as well as on $A$.

You should use indexing to do the operations.  E.g. `A[0, :]` refers to the whole first row of `A`, as does `A[0]`, so `A[1, :] = A[1, :] + 3*A[0, :]` would do the row operation $R_1 \leftarrow R_1 + 3R_0$, as would `A[1] = A[1] + 3*A[0]`.

Write a function `rowop` that performs a specified row operation on a given matrix and column vector:
```
def rowop(A, b, i, j, r):
    A = A.copy()
    b = b.copy()
    ...
    return A, b
```
Note: like lists and dictionaries, Numpy arrays are passed into functions by reference, not by value.  This means if you modify `A` inside `rowop` without making a copy first, you are actually modifying the original array.  Modifying things passed into functions is usually a bad idea, as it makes the code harder to reason about, and may trip you up.

The function `rowop` should take as arguments:
- `A`: an $n \times n$ matrix.
- `b`: an $n \times 1$ column vector.
- `i`, `j`: row numbers between $0$ and $n-1$.
- `r`: a real number.

The function should perform the row operation

$$
  R_i \leftarrow R_i + r R_j
$$

on the matrix `A` and vector `b`.  The new matrix and vector are then returned by the function.

**Hint:** You should use indexing to operate on the whole row of `A` with a single command.

Check that your `rowop` function works by defining `A` and `b` as above and running
```
  new_A, new_b = rowop(A, b, 1, 0, -1)
```
which should give

$$
  \mathtt{new}\_\mathtt{A} = \begin{bmatrix} 2 & \phm4 & -1 \\ 0 & -2 & \phm9 \\ 1 & \phm1 & \phm9 \end{bmatrix}
  \!\qquad \text{and} \qquad
  \mathtt{new}\_\mathtt{b} = \begin{bmatrix} -2 \\ \phm3 \\ \phm0 \end{bmatrix}.
$$


In [11]:
import numpy as np

def rowop(A,b,i,j,r):
    A = A.copy()
    b = b.copy()

    b[i] = b[i] - (A[i,j]/A[j,j])*b[j]
    A[i,:] = A[i,:] - (A[i,j]/A[j,j])*A[j,:]
    
    return A, b

A = np.array([[1,2],[3,4]])
b = np.array([[1],[2]])
rowop(A, b, 1, 0, 2)

(array([[ 1,  2],
        [ 0, -2]]),
 array([[ 1],
        [-1]]))

`rowop` is now a useful building block, and we won't modify it again.  Below, write code that defines `A` and `b` and uses your `rowop` function three times, to perform the above row operation and two more to reduce $A$ to an upper-triangular matrix (i.e. all entries below the main diagonal are zero).

In [ ]:
type your code here

Now write a function `gauss_elim3` that takes in a matrix `A` and right hand side vector `b` and calls `rowop` to perform the (three) required row operations to reduce a general $3 \times 3$ matrix to upper-triangular form.

In [20]:
import numpy as np

def rowop(A,b,i,j,r):
    A = A.copy()
    b = b.copy()

    c = (A[i,j]/A[j,j])
    b[i] = b[i] - c*b[j]
    A[i,:] = A[i,:] - c*A[j,:]
    
    return A, b

def gauss_elem3(A, b):
    for j in range(0,3):
        for i in range(j+1,3):
            A, b = rowop(A, b, i, j, 0)
    return A, b
            
A = np.array([[1,2,3],[2,3,4],[3,4,5]])
b = np.array([[1],[2],[3]])
gauss_elem3(A, b)

(array([[ 1,  2,  3],
        [ 0, -1, -2],
        [ 0,  0,  0]]),
 array([[1],
        [0],
        [0]]))

## Back substitution

The next step in solving the linear system is to find $\vx$ by back substitution.  Below, write a `back_sub3` function that takes in a row-reduced `A` and `b` and returns the solution $\vx$.
```
def back_sub3(A, b):
    ...
    return x
```

To calculate $\vx$ by back substitution, start by defining `x` as a $3 \times 1$ vector of zeros, and then calculating the last element of $\vx$, `x[2]`:
```
  x = np.zeros((3, 1))
  x[2] = b[2]/A[2, 2]
```
Now calculate `x[1]` and then `x[0]`.  **Hint:** The command for `x[1]` is NOT: `x[1] = b[1]/A[1, 1]`.  If you can't remember how to do back substitution, **try doing it on paper**, or look at your EMTH118 lecture notes from last year!

Test your `back_sub3` function on the results of your `gauss_elim3` function above.

In [43]:
import numpy as np
def back_sub3(A,b):
    A = A.copy()
    b = b.copy()
    
    x = np.zeros((3,1))
    
    x[2] = b[2]/A[2,2]
    x[1] = (b[1] - x[2]*A[1,2])/A[1,1]
    x[0] = (b[0] - x[2]*A[0,2] - x[1]*A[0,1])/A[0,0]
    
    return x
  
A = np.array([[1,2,4],[2,3,4],[3,6,12]])
b = np.array([[1],[2],[3]])
back_sub3(A, b)  

array([[-0.66666667],
       [ 0.33333333],
       [ 0.25      ]])

Below, write a `gauss3_basic` function that uses your `gauss_elim3` and `back_sub3` functions and returns the solution $\vx$:
```
def gauss3_basic(A, b):
    ...
    return x
```

Incorporate these commands into your `gauss3_basic` function to find $\vx$ by back substitution.  Your functions should not display anything.

<div class="alert alert-warning">
  <h3 style="margin-top: 0;">Checkpoint 1</h3>

  Use <code>gauss3_basic</code> to solve this system $A\vx = \vb$ where $A$ is a random $3 \times 3$ matrix and $\vb$ is a vector of $1$s.  You can define $A$ and $\vb$ with the commands
<pre><code>A = np.random.rand(3, 3)
b = np.ones(3)
</code></pre>
  Check that the answer given by your function is correct (think about how you can do this).
</div>

In [27]:
import numpy as np

def rowop(A,b,i,j,r):
    A = A.copy()
    b = b.copy()

    c = (A[i,j]/A[j,j])
    b[i] = b[i] - c*b[j]
    A[i,:] = A[i,:] - c*A[j,:]
    
    return A, b

def gauss_elem3(A, b):
    for j in range(0,3):
        for i in range(j+1,3):
            A, b = rowop(A, b, i, j, 0)
    return A, b

def back_sub3(A,b):
    A = A.copy()
    b = b.copy()
    
    x = np.zeros((3,1))
    
    x[2] = b[2]/A[2,2]
    x[1] = (b[1] - x[2]*A[1,2])/A[1,1]
    x[0] = (b[0] - x[2]*A[0,2] - x[1]*A[0,1])/A[0,0]
    
    return x

def rand_A_b():
    b = np.ones(3)
    A = np.random.rand(3,3)
    return A, b

A = np.array([[1,2,4],[2,3,4],[1,2,16]])
b = np.array([[1],[2],[3]])

# A, b = rand_A_b()

A, b = gauss_elem3(A, b)

x = back_sub3(A,b)

print(x)


[[ 1.66666667]
 [-0.66666667]
 [ 0.16666667]]


## Problems with the basic method
Now try using `gauss3_basic` to solve $A\vx = \vb$ where $\vb$ is a vector of $1$s and

$$
  \textrm{(i)} \qquad
  A = \begin{bmatrix}
        \phm0 & \phm1 & \phm2 \\
        \phm1 & -1 & \phm8 \\
        -4 & -1 & \phm3 \\
      \end{bmatrix},
  \qquad \qquad
  \textrm{(ii)} \qquad
  A = \begin{bmatrix}
        2 & \phm2 & -1 \\
        2 & \phm2 & \phm8 \\
        1 & -1 & \phm9 \\
      \end{bmatrix}.
$$

What do you think went wrong?

Reminder: Make sure your arrays are arrays of `float`s.

In [ ]:
type your code here

Below, make a copy of `gauss_elim3` called `gauss_elim_pivot3` and modify it so that it can cope with this type of problem.  Make a copy of `gauss3_basic` called `gauss3` that uses the new `gauss_elim_pivot3` function.

**Hint:** You can index with lists, not just numbers, so e.g. `A[[0, 2]]` will give you rows 0 and 2.  This makes swapping rows easy: `A[[0, 1]] = A[[1, 0]]` will swap the first two rows.

**Hint 2:** You may find `np.argmax` useful.  It returns the index of the maximum element.  Alternatively, you can find the maximum location with `if` and `>`.

<div class="alert alert-warning">
  <h3 style="margin-top: 0;">Checkpoint 2</h3>

  Use your new function to solve these two systems $A\vx = \vb$.  Check that the answer given by your function is correct.
</div>

In [1]:
import numpy as np

def rowop(A,b,i,j,r):
    A = A.copy()
    b = b.copy()

    c = (A[i,j]/A[j,j])
    b[i] = b[i] - c*b[j]
    A[i,:] = A[i,:] - c*A[j,:]
    
    return A, b

def gauss_elem_pivot3(A, b):
    for j in range(0,3):
        max_pivot = -1

        for a in range(j,3):
            if np.abs(A[a,j]) > max_pivot:
                max_pivot = np.abs(A[a,j])
                index = a

        A[[j, index]] = A[[index,j]]
        
        for i in range(j+1,3):
            A, b = rowop(A, b, i, j, 0)
    return A, b

def back_sub3(A,b):
    A = A.copy()
    b = b.copy()
    
    x = np.zeros((3,1))
    
    x[2] = b[2]/A[2,2]
    x[1] = (b[1] - x[2]*A[1,2])/A[1,1]
    x[0] = (b[0] - x[2]*A[0,2] - x[1]*A[0,1])/A[0,0]
    
    return x

def rand_A_b():
    b = np.ones(3)
    A = np.random.rand(3,3)
    return A, b

A, b = rand_A_b()

A, b = gauss_elem_pivot3(A, b)

x = back_sub3(A,b)

print(x)


[[-9.11518626e-04]
 [ 1.37103244e+00]
 [ 2.15608324e-01]]


## General systems of $n$ linear equations
Below, make copies of your original `gauss_elim3`, `back_sub3`, and `gauss3_basic` functions (the versions without pivoting) called `gauss_elim`, `back_sub`, and `gauss_basic`, and modify them so that `gauss_basic` can solve a general system of $n$ linear equations, where $n$ can be any positive integer.  (You don't need to worry about pivoting here.)

**Hint:** you will need to use two nested `for` loops to do the elimination process, followed by another two nested `for` loops for the back substitution process.  Think about which matrix entries you would need to ‘eliminate’ (i.e. make into zeros), and the order in which you would eliminate them, if you were doing Gaussian elimination on a large matrix.  Your first set of nested `for` loops should go through these entries in the same order, performing the necessary row operation at each step to eliminate that entry.  Your second set of nested loops should perform a similar procedure for back substitution.

<div class="alert alert-warning">
  <h3 style="margin-top: 0;">Checkpoint 3</h3>

  Use your code to solve $A\vx = \vb$ where $A$ is a random $10 \times 10$ matrix and $\vb$ is a vector of $1$s.  Check that the answer given by your function is correct.
</div>

In [14]:
import numpy as np

def rowop(A,b,i,j,r):
    A = A.copy()
    b = b.copy()

    c = (A[i,j]/A[j,j])
    b[i] = b[i] - c*b[j]
    A[i,:] = A[i,:] - c*A[j,:]
    
    return A, b

def gauss_elem_pivot3(A, b):
    for j in range(0,3):
        max_pivot = -1

        for a in range(j,3):
            if np.abs(A[a,j]) > max_pivot:
                max_pivot = np.abs(A[a,j])
                index = a

        A[[j, index]] = A[[index,j]]
        
        for i in range(j+1,3):
            A, b = rowop(A, b, i, j, 0)
    return A, b

def gauss_elem3(A, b):
    for j in range(0,len(A)):
        for i in range(j+1,len(A[:])):
            A, b = rowop(A, b, i, j, 0)
    return A, b

def back_sub3(A,b):
    A = A.copy()
    b = b.copy()
    x = np.zeros((len(A),1))
    s = 0
    for i in range(len(A)-1, -1, -1):
        for j in range(len(A)-1, i-1, -1):
            s = s + A[i,j]*x[j]
        x[i] = (b[i]-s)/A[i,i]
    return x

def rand_A_b():
    n = 6
    b = np.ones(n)
    A = np.random.rand(n,n)
    return A, b

# A, b = rand_A_b()

A = np.array([[1,2,3],[2,3,4],[4,5,5]])
b = np.array([[1],[2],[3]])

A, b = gauss_elem3(A, b)
print(A)
x = back_sub3(A,b)

print(x)


[[ 1  2  3]
 [ 0 -1 -2]
 [ 0  0 -1]]
[[ 4.]
 [-2.]
 [ 1.]]
